# The Main Characteristics of Road Collisions in the UK for 2023

## About the analysis

This analysis aims to explore the circumstances of UK road collisions in the year 2023 based on Road Safety data publicly available on [data.gov.uk](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data).

What were the main characteristics and possible contributing factors to road collisions in 2023? Are there any patterns for when and where most collisions occurred? What other potential elements, such as speed, road type or weather conditions, were at play? Is there a correlation between these elements and the severity of accidents?

It is crucial to have an overview and gain a good understanding of the potential main contributing factors of these events to help prevent as many accidents in the future as possible.

_The form used to gather the data can be found [here](https://assets.publishing.service.gov.uk/media/60d0cc548fa8f57ce4615110/stats19.pdf), and the instruction manual containing the collection criteria and guidance is available [here](https://assets.publishing.service.gov.uk/media/60d0cc968fa8f57cf3f0b3ad/stats20-2011.pdf). The latter will be referenced in this notebook to aid understanding around specifics of the data._

## Importing libraries and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import seaborn as sns

# mapping
import folium
from folium.plugins import HeatMap, MarkerCluster, Fullscreen, TagFilterButton
import requests

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# display all columns in dataframe & set default Seaborn theme
pd.set_option('display.max_columns', None)
sns.set_theme(style="darkgrid", palette='mako')

In [ ]:
# importing data
df = pd.read_csv('data/dft-road-casualty-statistics-collision-2023.csv',
                 dtype={'accident_index': object, 'accident_reference': object})

# importing data guide containing labels for numerical values
labels = pd.read_excel('data/dft-road-casualty-statistics-road-safety-open-dataset-data-guide-2023.xlsx',
                      dtype={'table': object, 'field_name': object, 'code/format': object, 'label': object, 'note': object})

# importing estimated population data
population = pd.read_excel('data/population_estimates_mid2023_england_wales.xlsx')

# converting date column to datetime format, adding year/month/day as separate columns
df['date'] = df['date'] + ' ' + df['time']
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df.drop(columns=['time'], inplace=True)
df['time'] = df['date'].dt.time

## Exploring and cleaning data

In [ ]:
print('Rows: ' + str(df.shape[0]))
print('Columns: ' + str(df.shape[1]) + '\n')
display(df.head(3))

In [ ]:
# dropping unwanted columns

df.drop(columns=['accident_year', 'location_easting_osgr', 'location_northing_osgr',
                 'lsoa_of_accident_location', 'local_authority_district', 'second_road_class',
                 'first_road_number', 'second_road_number', 'enhanced_severity_collision', 'trunk_road_flag'
                ], inplace=True)

# enhanced_severity_collision data was only starting to be collected later in the year

display(df.info())
display(df.describe())

In [ ]:
# checking missing values

df.isna().sum()

In [ ]:
# dropping 12 columns where location is unidentified (0.01% of data)

df.dropna(inplace = True, ignore_index=True)
df.index

### Assigning labels

In [ ]:
# sample of label data

labels.sample(n=5, random_state=1)

In [ ]:
# cleaning unknowns before applying labels

df['special_conditions_at_site'] = df['special_conditions_at_site'].map(lambda x: 9 if x == 0 or x == -1 else x)
df['road_surface_conditions'] = df['road_surface_conditions'].map(lambda x: -1 if x == 9 else x)
df['pedestrian_crossing_physical_facilities'] = df['pedestrian_crossing_physical_facilities'].map(lambda x: -1 if x == 9 else x)
df['pedestrian_crossing_human_control'] = df['pedestrian_crossing_human_control'].map(lambda x: -1 if x == 9 else x)
df['junction_control'] = df['junction_control'].map(lambda x: -1 if x == 9 else x)
df['junction_detail'] = df['junction_detail'].map(lambda x: -1 if x == 99 else x)
df['carriageway_hazards'] = df['carriageway_hazards'].map(lambda x: -1 if x == 0 or x == 9 else x)

# list of columns that do not need labels

label_exceptions = ['accident_index', 'accident_reference', 'latitude', 'longitude', 'number_of_vehicles','number_of_casualties',
                   'date', 'time', 'speed_limit', 'year', 'month', 'day', 'day_of_week']

# loop through columns, filter labels to current column labels, create dict
# map column to labels in dict

for col in df.columns:
    if col not in label_exceptions:

        filter = labels[labels['field_name'] == col].reset_index()
        label_dict = dict(zip(filter['code/format'], filter['label']))
        df[col+'_labels'] = df[col].map(label_dict).astype('category')


In [ ]:
# checking NaN values in new label columns

label_cols = [col for col in df.columns if 'label' in col]

df[label_cols].isna().sum()

In [ ]:
# the week starts with Sunday in data, fixing this in the numerical column first

df['day_of_week'] = df['day_of_week'].map(lambda x: x-1 if x != 1 else 7)

# creating labels for weekdays to use for visualisation

df['day_of_week_labels'] = df['date'].dt.day_name()

### Checking potential outliers and curiosities in data

In [ ]:
# max number of vehicles involved

print('\n')
print('Max number of vehicles involved in a collision: ' + str(df['number_of_vehicles'].max()) + '\n')

veh_17_mask = df[df['number_of_vehicles'] == df['number_of_vehicles'].max()]
display(veh_17_mask)
print('\n')

# highest casualty number

print('\n')
print('Max number of casualties: ' + str(df['number_of_casualties'].max()) + '\n')

casualty_70_mask = df[df['number_of_casualties'] == df['number_of_casualties'].max()]
display(casualty_70_mask)
print('\n')

# any fatal accidents police did not attend?

print('\n')
df_severity_mask = df[df['accident_severity'] == 1]
display(df_severity_mask['did_police_officer_attend_scene_of_accident_labels'].value_counts())
print('\n')

## Data Exploration

Looking at proportions, distribution and frequency of accident severity and exploring relations with a heatmap.

**Fatal injury:**
Cases where death occurs in less than 30 days as a result of the accident.

**Serious injury:**
Injuries resulting in hospitalisation and/or long term consequences, i.e. severe head or chest injury, internal injuries, fractures, concussion.

**Slight injury:**
Minor injuries i.e. whiplash, neck pain, shallow cuts, bruising.

In [ ]:
df['accident_severity_labels'].value_counts()

plt.figure(figsize=(6, 6))
plt.pie(data=df,
        x=df['accident_severity'].value_counts(),
        labels=df['accident_severity_labels'].unique(),
        autopct='%1.1f%%',
        pctdistance=1.15,
        labeldistance=1.3,
        colors=[sns.color_palette()[2],sns.color_palette()[1],sns.color_palette()[0]]
       )

plt.title('Accident Type Proportions', fontweight='bold')

plt.show();

In [ ]:
plt.figure(figsize=(6, 6))
sns.boxplot(data=df, x='accident_severity_labels', y='speed_limit', hue='accident_severity_labels')

plt.title('Distribution of Accidents by Severity', fontweight='bold')
plt.xlabel('Accident severity')
plt.ylabel('Speed limit')

plt.show();

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.countplot(
    data=df,
    x='accident_severity_labels',
    hue='accident_severity_labels'
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container)

plt.title('Frequency of Accidents by Severity', fontweight='bold')
plt.xlabel('Accident severity')
plt.ylabel('Frequency')

plt.show();

In 2023 there were 286 accidents on average daily, 4 of which were fatal.

In [ ]:
# removing year column as current analysis only involves 1 year and selecting numerical datatypes
heat_cols = [col for col in df.columns if 'year' not in col]
explore_corr = df[heat_cols].select_dtypes(include='number').corr()

f, ax = plt.subplots(figsize=(15, 10))

ax = sns.heatmap(
    data = explore_corr,
    cmap = sns.diverging_palette(220, 20, as_cmap=True),
    annot = True,
    fmt = '.1f',
    linewidths = .3,
    cbar_kws = {"shrink": .7},
    vmin = -1,
    vmax = 1,
    mask = np.triu(explore_corr)
)

ax.grid(False)
plt.title('Correlation Heatmap', fontweight='bold', fontsize=20)

plt.show()

## Data Analysis

### Time Analysis

Analysing the data to check patterns and trends through time. Analysing accidents in relation to months, days of week and hours of day.

In [ ]:
months_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

g = sns.displot(
    data=df,
    x='month',
    hue='accident_severity_labels',
    multiple='dodge', # layer or stack, alternatively
    edgecolor='.3',
    linewidth=.3,
    discrete=True,
    aspect=2,
    stat='count'
)


# set graph parameters i.e. labels, legend, etc.
g.set(xticks=range(1,13,1))
g.set_xticklabels(months_order)
plt.title('Frequency of Accidents by Month', fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Frequency of Accidents')
sns.move_legend(g, 'lower center', bbox_to_anchor=(0.44, 0.9), ncol=3, frameon=False, title=None)

plt.show();

In [ ]:
# adding jitter to x and y axes
df_jitter = df[['month', 'number_of_vehicles', 'accident_severity_labels']]
df_jitter['month'] = df_jitter['month'] + np.random.normal(0, 0.15, size=len(df['month']))
df_jitter['month'] = df_jitter['month'].apply(lambda x: 0 if x < 0 else x)
df_jitter['number_of_vehicles'] = df_jitter['number_of_vehicles'] \
+ np.random.normal(0, 0.3, size=len(df_jitter['number_of_vehicles']))
df_jitter['number_of_vehicles'] = df_jitter['number_of_vehicles'].apply(lambda x: 0 if x < 0 else x)


plt.figure(figsize=(10, 5))
g = sns.scatterplot(
    data=df_jitter,
    x='month',
    y='number_of_vehicles',
    hue='accident_severity_labels',
    alpha=0.5
)

plt.axhline(df_jitter['number_of_vehicles'].mean(), color='gold', linestyle=':', linewidth=2)

# set graph parameters
g.set_xticklabels(months_order)
g.set(xticks=range(1,13,1))
g.set(yticks=range(0,18,1))
plt.title('Number of Vehicles Involved by Month', fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Vehicles Involved')
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 0.65), title=None)

plt.show();

In [ ]:
# adding jitter to x and y axes
df_jitter_casualties = df[['month', 'number_of_casualties', 'accident_severity_labels']]
df_jitter_casualties = df_jitter_casualties[df_jitter_casualties['number_of_casualties'] < 20]
df_jitter_casualties['month'] = df_jitter_casualties['month'] + np.random.normal(0, 0.2, size=len(df_jitter_casualties['month']))
df_jitter_casualties['month'] = df_jitter_casualties['month'].apply(lambda x: 0 if x < 0 else x)
df_jitter_casualties['number_of_casualties'] = df_jitter_casualties['number_of_casualties'] \
+ np.random.normal(0, 0.3, size=len(df_jitter_casualties['number_of_casualties']))
df_jitter_casualties['number_of_casualties'] = df_jitter_casualties['number_of_casualties'].apply(lambda x: 0 if x < 0 else x)


plt.figure(figsize=(10, 5))
g = sns.scatterplot(
    data=df_jitter_casualties,
    x='month',
    y='number_of_casualties',
    hue='accident_severity_labels',
    alpha=0.5
)

plt.axhline(df_jitter_casualties['number_of_casualties'].mean(), color='gold', linestyle=':', linewidth=2)

# set graph parameters
g.set_xticklabels(months_order)
g.set(xticks=range(1,13,1))
g.set(yticks=range(0,21,1))
plt.title('Number of Casualties by Month', fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Casualties')
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 0.65), title=None)

plt.show();

In [ ]:
weekdays_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.figure(figsize=(10, 4))
sns.countplot(
    data=df,
    x='day_of_week_labels',
    hue='day_of_week_labels',
    order=weekdays_order,
    hue_order=weekdays_order,
    stat='probability',
    palette=sns.color_palette()
)

plt.title('Probability of Accidents by Day of Week', fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Probability')

plt.show();

In [ ]:
# adding jitter to y axis
df_jitter_day = df[['day_of_week', 'number_of_vehicles', 'accident_severity_labels']]
df_jitter_day['number_of_vehicles'] = df_jitter_day['number_of_vehicles'] + np.random.normal(0, 0.3, size=len(df['number_of_vehicles']))

# preventing negative and very high values (outliers)
df_jitter_day['number_of_vehicles'] = df_jitter_day['number_of_vehicles'].apply(lambda x: 0 if x < 0 else x)
df_jitter_day = df_jitter_day[df_jitter_day['number_of_vehicles'] <= 6]

# taking a sample of 1000 for performance reasons
df_jitter_day_sample = df_jitter_day.sample(frac=0.01, random_state=1)


plt.figure(figsize=(10, 5))
g = sns.swarmplot(
    data=df_jitter_day_sample,
    x='day_of_week',
    y='number_of_vehicles',
    hue='accident_severity_labels',
    alpha=0.7
)

# plt.axhline(df_jitter_day_sample['number_of_vehicles'].mean(), color='black', linestyle='--', linewidth=2)

# set graph parameters
g.set_xticklabels(weekdays_order)
g.set(yticks=range(0,7,1))
plt.title('Number of Vehicles Involved by Day of Week (Sample of 1000)', fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Number of Vehicles Involved')
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 0.65), title=None)

plt.show();

In [ ]:
hours = df[['accident_index', 'date', 'day_of_week', 'day_of_week_labels']]

# converting timestamps to useable format for plotting
hours['hours_minutes'] = hours['date'].dt.time
hours['hours'] = hours['date'].values.astype('<M8[h]')
hours['hours'] = hours['hours'].dt.time.astype(str)
hours['hours'] = hours['hours'].apply(lambda x: x[:-3])
hours

# grouping values and adding count
hours_group = hours.groupby(['day_of_week', 'hours'])['accident_index'].size().reset_index(name='count')

In [ ]:
# defining colour palette
a = 'midnightblue'
b = 'darkturquoise'
c = 'dodgerblue'
d = 'orangered'
e = 'purple'

palette = a,a,a,a, a,a,a,b, b,b,b,c, c,c,c,d, d,d,d,e, a,a,a,a
# palette=sns.color_palette("coolwarm", 12) + sns.color_palette("coolwarm_r", 12)
# 'blend:midnightblue,gold,mediumseagreen,orangered,midnightblue'


g = sns.catplot(
    data=hours_group,
    x='day_of_week',
    y='count',
    hue='hours',
    kind='swarm',
    aspect=1.4,
    height=5.7,
    zorder=1,
    palette=palette
)

g.set_xticklabels(weekdays_order)


sns.regplot(
    data=hours_group,
    x='day_of_week',
    y='count',
    scatter=False,
    truncate=False,
    order=2,
    color='.2'
)

plt.title('Frequency of Accidents by Hour', fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Accident Frequency')

plt.show();

In [ ]:
# converting timestamps to useable format for plotting
hours_group['hours_int'] = hours_group['hours'].apply(lambda x: int(x[0:2]))
hours_heat = hours_group[['day_of_week', 'hours_int', 'count']]
hours_heat = hours_heat.pivot(index='hours_int', columns='day_of_week', values='count')
hours_heat

plt.figure(figsize=(10, 8))
g = sns.heatmap(
    data=hours_heat,
    linewidths=.01,
    linecolor='black',
    cmap=sns.color_palette(),
    annot=True,
    fmt='.0f',
    annot_kws={'size':8},
    norm=BoundaryNorm([0,300,600,900,1200,1500], ncolors=6)
)

# set graph parameters
g.set_xticklabels(weekdays_order)
plt.title('Heatmap of Accident Frequency', fontweight='bold')
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')
plt.yticks(rotation=0)

plt.show();

In [ ]:
# converting timestamps to useable format for plotting
hours_dt_group = hours_group
hours_dt_group['datetime'] = hours_group['datetime'] = '1970-01-01'
hours_dt_group['datetime'] = hours_dt_group['datetime'] + ' ' + hours_dt_group['hours'].astype(str)
hours_dt_group['datetime']= pd.to_datetime(hours_dt_group['datetime'])


g = sns.displot(
    data=hours_dt_group,
    x='count',
    hue='day_of_week',
    kind='kde',
    height=6,
    multiple='fill',
    legend=False,
    palette=sns.color_palette()
)

# set graph parameters
plt.legend(title='Day of Week', loc='upper right', bbox_to_anchor=(1.3, 0.7), labels=weekdays_order)
plt.title('Accident Density by Weekday', fontweight='bold')
plt.xlabel('Frequency')
plt.ylabel('Density')
plt.xlim(0)
plt.ylim(0)

plt.show();

In [ ]:
# converting timestamps to useable format for plotting
hours = df[['accident_index', 'date', 'day_of_week', 'day_of_week_labels']]
hours['hours_numerical'] = hours['date'].dt.hour

hours_days = hours[['day_of_week_labels', 'hours_numerical']]
hours_days = hours_days.groupby(['day_of_week_labels', 'hours_numerical'])['hours_numerical'].count().reset_index(name='count')


g = sns.relplot(
    data=hours_days,
    x='hours_numerical',
    y='count',
    col='day_of_week_labels',
    hue='day_of_week_labels',
    kind='line',
    linewidth=4,
    col_order=weekdays_order,
    hue_order=weekdays_order,
    col_wrap=2,
    height=2.5,
    aspect=1.5,
    legend=False,
    palette=sns.color_palette()
)

# loop through subplots and add title for each
for day, ax in g.axes_dict.items():
    ax.text(.4,.1, day, transform=ax.transAxes, fontweight='bold')


# set x and y labels
g.set(xticks=range(0,25,4))
g.set(yticks=range(0,1501,500))
plt.xlim(0)
plt.ylim(0)

g.fig.suptitle('Accidents by Hour for Days of Week', fontweight='bold')
g.set_titles('')
g.set_axis_labels('Hour of Day', 'Accidents (cumulative)')

plt.show();

### Location Analysis

Analysing the data to discover geospatial patterns and trends.

In [ ]:
# loading in population data
population.columns = population.columns.str.lower()
population.columns = population.columns.str.replace(' ', '_')

# applying data guide labels on code column of population data
# not using name column to easily drop unwanted rows

district_filter = labels[labels['field_name'] == 'local_authority_ons_district'].reset_index()
district_dict = dict(zip(district_filter['code/format'], district_filter['label']))
population['ons_district'] = population['code'].map(district_dict)
population = population.dropna().reset_index()

population_all_ages = population[['code', 'ons_district', 'all_ages']]

# creating data excerpts and merging data

df_population_excerpt = df[['local_authority_ons_district', 'local_authority_ons_district_labels']]
df_population_excerpt.rename(columns={'local_authority_ons_district' : 'code',
                                    'local_authority_ons_district_labels' : 'ons_district'},
                          inplace=True)

population_count = df_population_excerpt.groupby(['code']).size().reset_index(name='accidents_per_district')
population_all_ages = population_all_ages.merge(population_count, how='inner', on='code')

population_all_ages['accident_ratio'] = population_all_ages['accidents_per_district']

for i in population_all_ages.index:
    population_all_ages['accident_ratio'][i] = \
    (population_all_ages['accident_ratio'][i] / population_all_ages['all_ages'][i] * 100).round(2)

# population_all_ages

In [ ]:
top5_total = population_all_ages.sort_values(by='accidents_per_district', ascending=False)[:5]

plt.figure(figsize=(9, 6))
sns.barplot(
    data=top5_total,
    x='ons_district',
    y='accidents_per_district',
    palette=sns.color_palette()
)

plt.title('Top 5 Accident-Prone Districts by Accident Frequency', fontweight='bold')
plt.xlabel('District')
plt.ylabel('Accident Frequency')

plt.show;



ratio_top5 = population_all_ages.sort_values(by='accident_ratio', ascending=False)[:5]

plt.figure(figsize=(9, 6))
sns.barplot(
    data=ratio_top5,
    x='ons_district',
    y='accident_ratio',
    palette=sns.color_palette()
)

plt.title('Top 5 Accident-Prone Districts by Population', fontweight='bold')
plt.xlabel('District')
plt.ylabel('Accident Ratio')

plt.show();

In [ ]:
labels = 'local_authority_highway_labels'
highway_top10 = df.groupby(labels).size().reset_index(name='count')
highway_top10 = highway_top10.sort_values(by='count', ascending=False)[:10]
highway_top10[labels] = highway_top10[labels].astype(str)

plt.figure(figsize=(12, 6))
sns.barplot(
    data=highway_top10,
    x=labels,
    y='count',
    palette=sns.color_palette()
)

plt.title('Top 10 Highway Authorities with Most Accidents', fontweight='bold')
plt.xlabel('Highway Authority')
plt.ylabel('Accident Frequency')

plt.show();

In [ ]:
labels = 'police_force_labels'
police_force_top5 = df.groupby(labels).size().reset_index(name='count')
police_force_top5 = police_force_top5.sort_values(by='count', ascending=False)[:5]
police_force_top5[labels] = police_force_top5[labels].astype(str)

plt.figure(figsize=(12, 6))
sns.barplot(
    data=police_force_top5,
    x=labels,
    y='count',
    palette=sns.color_palette()
)

plt.title('Top 5 Police Force Attending', fontweight='bold')
plt.xlabel('Police Force')
plt.ylabel('Frequency')

plt.show();

In [ ]:
# UK heatmap

# map parameters

m = folium.Map(location=[54,-1],
               zoom_start=5,
               tiles='cartodbpositron',
               min_zoom=5,
               min_lat=49.5,
               max_lat=61,
               min_lon=-10,
               max_lon=3,
               max_bounds=True
              )


# fullscreen feature

folium.plugins.Fullscreen(
    position="topright",
    title="Expand",
    title_cancel="Exit",
    force_separate_button=True,
).add_to(m)


# heatmap parameters

HeatMap(data=df[['latitude', 'longitude']], radius=13).add_to(m)
m

In [ ]:
# UK fatal accidents cluster map

# folium TagFilterButton plugin works, but does not clear markers on whole map, so separate map created for fatal collisions
# additional information added to markers

# data excerpt

df_fatal_cluster = df[['latitude', 'longitude', 'accident_severity', 'date', 'time', 'road_type_labels', 'speed_limit',
                      'number_of_vehicles', 'number_of_casualties']]
df_fatal_cluster = df_fatal_cluster[df_fatal_cluster['accident_severity'] == 1]


# map parameters

tiles = 'https://{s}.tile.thunderforest.com/neighbourhood/{z}/{x}/{y}.png?apikey=311627def05c41749ec8a633497a6e38' # 'cartodbpositron'
attr = ('&copy; <a href="http://www.thunderforest.com/">Thunderforest</a>, &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

m = folium.Map(location=[54,-1],
               zoom_start=5,
               tiles=tiles,
               attr=attr,
               min_zoom=5,
               min_lat=49.5,
               max_lat=61,
               min_lon=-10,
               max_lon=3,
               max_bounds=True
              )


# fullscreen feature

folium.plugins.Fullscreen(
    position="topright",
    title="Expand",
    title_cancel="Exit",
    force_separate_button=True,
).add_to(m)


# markers

mc = MarkerCluster().add_to(m)

for id, row in df_fatal_cluster.iterrows():

    speed = row['speed_limit']
    road = row['road_type_labels']
    date = row['date']
    vehicles = row['number_of_vehicles']
    casualties = row['number_of_casualties']
    
    desc = f'''
    <b>Vehicles:</b> {vehicles} <br>
    <b>Casualties:</b> {casualties} <br>
    <b>Speed limit:</b> {speed} <br>
    <b>Road type:</b> {road} <br>   
    <b>Date:</b> {date} <br>
    '''
    
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = folium.Popup(desc, parse_html=False, max_width=1000),
        icon = folium.Icon(color='red', icon='remove-sign')
    ).add_to(mc)

m

# m.save("cluster_map.html")

In [ ]:
# England and Wales accident ratio across districts map

# setting up map properties
m = folium.Map(location=[54,-1],
               zoom_start=6,
               tiles='cartodbpositron',
               min_zoom=5,
               min_lat=49.5,
               max_lat=61,
               min_lon=-10,
               max_lon=3,
               max_bounds=True
              )

# fullscreen feature
folium.plugins.Fullscreen(
    position="topright",
    title="Expand",
    title_cancel="Exit",
    force_separate_button=True,
).add_to(m)


# retrieve GeoJSON -- online version may not always load, alternatively use local version

uk_districts = requests.get(
    'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_December_2023_Boundaries_UK_BFC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
).json()


# creating choropleth map
cp = folium.Choropleth(
    geo_data=uk_districts, # 'data/Local_Authority_Districts_December_2023_Boundaries_UK_BFC.geojson' -- local file
    data=population_all_ages,
    columns=['code','accident_ratio'],
    key_on='feature.properties.LAD23CD',
    bins=13,
    smooth_factor=2,
    fill_opacity=1,
    line_weight=0.3,
    legend_name='Accident Ratio (%)',
    highlight=True,
    nan_fill_color='grey',
    nan_fill_opacity=0.2,
    fill_color='Reds' # YlOrRd, BuPu
).add_to(m)


# setting up tooltips

# index data to LAD23CD codes
population_all_ages_indexed = population_all_ages.set_index('code')

# loop through geojson and add accident ratio to properties
for row in cp.geojson.data['features']:
    try:
        row['properties']['accident_ratio'] = population_all_ages_indexed.loc[row['properties']['LAD23CD'], 'accident_ratio']
    except KeyError:
        row['properties']['accident_ratio'] = 'No data'

# customising tooltips
folium.GeoJsonTooltip(
    fields=['LAD23NM', 'accident_ratio'],
    aliases=['District:', 'Accident ratio:']
).add_to(cp.geojson)

m

### Environmental Factors

Checking the rest of the data for any environmental factors and conditions.

In [ ]:
labels = 'urban_or_rural_area_labels'
subdf = pd.DataFrame(df[labels].value_counts()).reset_index()


fig, ax = plt.subplots(1, 2, figsize=(11,5))

ax[0].pie(data=subdf,
        x=subdf['count'],
        labels=subdf[labels],
        autopct='%1.1f%%',
        pctdistance=1.15,
        labeldistance=1.3,
        colors=[sns.color_palette()[1],sns.color_palette()[2]],
)

ax[0].set_title('Accident Proportion by Area Type', fontweight='bold')



subdf = df[df['accident_severity'] == 1]
subdf = pd.DataFrame(subdf[labels].value_counts()).reset_index()

ax[1].pie(data=subdf,
        x=subdf['count'],
        labels=subdf[labels],
        autopct='%1.1f%%',
        pctdistance=1.15,
        labeldistance=1.3,
        colors=[sns.color_palette()[2],sns.color_palette()[1]],
)

ax[1].set_title('Accident Proportion by Area Type - Fatal Accidents', fontweight='bold')

plt.show();

In [ ]:
subdf = pd.DataFrame(df['speed_limit'].value_counts()).reset_index()
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = [70,60,50,40,30,20]

plt.figure(figsize=(8, 6))
ax = sns.barplot(
    data=subdf,
    x='speed_limit',
    y='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Speed Limit', fontweight='bold')
plt.xlabel('Speed Limit')
plt.ylabel('Proportion (%)')

plt.show();


# fatal accidents require a separate graph due to proportion of data being much smaller in comparison
subdf = df[df['accident_severity'] == 1]
subdf = pd.DataFrame(subdf['speed_limit'].value_counts()).reset_index()
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = [70,60,50,40,30,20]


plt.figure(figsize=(8, 6))
ax = sns.barplot(
    data=subdf,
    x='speed_limit',
    y='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Speed Limit - Fatal Accidents', fontweight='bold')
plt.xlabel('Speed Limit')
plt.ylabel('Proportion (%)')

plt.show();

In [ ]:
labels = 'first_road_class_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
subdf = subdf[subdf['percent'] > 0]
order = list(subdf[labels])


plt.figure(figsize=(8, 6))
ax = sns.barplot(
    data=subdf,
    x=labels,
    y='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Road Class', fontweight='bold')
plt.xlabel('Road Class')
plt.ylabel('Proportion (%)')

plt.show();

In [ ]:
labels = 'road_type_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    x=labels,
    y='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Road Type', fontweight='bold')
plt.xlabel('Road Type')
plt.ylabel('Proportion (%)')

plt.show();

In [ ]:
labels = 'junction_detail_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf[labels] != 'Data missing or out of range']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
subdf = subdf[subdf['percent'] > 0]
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Junction Detail', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Junction Detail')

plt.show();

In [ ]:
labels = 'junction_control_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf[labels] != 'Data missing or out of range']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    x=labels,
    y='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Junction Control', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Junction Control')

plt.show();

# NOTE: 44% of data missing or unknown

In [ ]:
labels = 'pedestrian_crossing_human_control_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf[labels] != 'Data missing or out of range']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
# subdf = subdf[subdf['percent'] > 0]
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Pedestrian Crossing with Human Control', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Crossing Control')

plt.show();

# NOTE: 8% of data missing or unknown

In [ ]:
labels = 'pedestrian_crossing_physical_facilities_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf[labels] != 'Data missing or out of range']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
subdf = subdf[subdf['percent'] > 0]
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Pedestrian Crossing Physical Facilities', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Crossing Facilities')

plt.show();

# NOTE: 8% of data missing or unknown

**Instruction manual 1.20a and 1.20b:** pedestrian crossing only recorded when considered to be a factor in the accident

In [ ]:
labels = 'light_conditions_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = list(subdf[labels])


plt.figure(figsize=(8, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Light Conditions', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Light Conditions')

plt.show();

**Instruction manual 1.21:** "The distinction between 'street lights unlit' and 'no street lights' is made because it is important in assessing factors affecting accident rates."

In [ ]:
labels = 'road_surface_conditions_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf[labels] != 'Data missing or out of range']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = list(subdf[labels])


plt.figure(figsize=(8, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Road Surface Conditions', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Road Surface Conditions')

plt.show();

# NOTE: 3% of data missing or unknown

In [ ]:
labels = 'special_conditions_at_site_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf['special_conditions_at_site_labels'] != 'unknown (self reported)']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Special Conditions', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Special Conditions')

plt.show();




labels = 'special_conditions_at_site_labels'
subdf = df[df[labels] != 'unknown (self reported)']
subdf[labels] = subdf[labels].astype(str)

plt.figure(figsize=(10, 6))
ax = sns.countplot(
    data=subdf,
    x='accident_severity_labels',
    hue=labels,
    palette=sns.color_palette() + ['mediumaquamarine']
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container)

plt.title('Accident Frequency by Special Conditions', fontweight='bold')
plt.xlabel('Accident Severity')
plt.ylabel('Frequency')
plt.legend(title='Special Conditions')

plt.show();


# NOTE: 98% of data missing or unknown !

In [ ]:
labels = 'carriageway_hazards_labels'

subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf = subdf[subdf[labels] != 'Data missing or out of range']
subdf['percent'] = (subdf['count'] / sum(subdf['count']) *100).round(0)
order = list(subdf[labels])


plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=subdf,
    y=labels,
    x='percent',
    order=order,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.0f%%')

plt.title('Accident Proportion by Carriageway Hazards', fontweight='bold')
plt.xlabel('Proportion (%)')
plt.ylabel('Carriageway Hazards')

plt.show();




subdf = df[df[labels] != 'Data missing or out of range']
subdf[labels] = subdf[labels].astype(str)

plt.figure(figsize=(10, 6))
ax = sns.countplot(
    data=subdf,
    x='accident_severity_labels',
    hue=labels,
    palette=sns.color_palette()
)

# adding totals on top of the bars
for container in ax.containers:
    ax.bar_label(container)

plt.title('Accident Frequency by Carriageway Hazards', fontweight='bold')
plt.xlabel('Accident Severity')
plt.ylabel('Frequency')
plt.legend(title='Carriageway Hazards')

plt.show();


# NOTE: 98% of data missing or unknown !

**Instruction manual 1.25:** Only live animals are coded under "animal in carriageway", dead animals are coded under "other objects".

In [ ]:
labels = 'did_police_officer_attend_scene_of_accident_labels'
subdf = pd.DataFrame(df[labels].value_counts()).reset_index()
subdf[labels] = subdf[labels].replace({
    'No - accident was reported using a self completion  form (self rep only)' : 'No (Self-Reported)'
})

plt.figure(figsize=(6, 6))
plt.pie(data=subdf,
        x=subdf['count'],
        labels=subdf[labels],
        autopct='%1.1f%%',
        pctdistance=1.15,
        labeldistance=1.3,
        colors=[sns.color_palette()[1],sns.color_palette()[2], sns.color_palette()[3]]
)


plt.title('Police Attending Scene of Accident', fontweight='bold')

plt.show();

## Potential for further analysis

* Expanding data with casualties and vehicles data recorded in 2023
* Pulling in datasets for previous years for further comparison
* Further explore correlations to serious accidents

## Unused code

In [ ]:
# heatmap is better for this purpose

# sns.catplot(
#     data=hours_group,
#     x='day_of_week',
#     y='hours',
#     hue='count',
#     aspect=1,
#     height=6,
#     s=75
# )

# plt.show();

In [ ]:
# # UK cluster map

# # data excerpt for iteration

# df_cluster = df[['latitude', 'longitude', 'accident_severity']]


# # map parameters

# m = folium.Map(location=[54,-1],
#                zoom_start=5,
#                tiles='cartodbpositron',
#                min_zoom=5,
#                min_lat=49.5,
#                max_lat=61,
#                min_lon=-10,
#                max_lon=3,
#                max_bounds=True
#               )


# # fullscreen feature

# folium.plugins.Fullscreen(
#     position="topright",
#     title="Expand",
#     title_cancel="Exit",
#     force_separate_button=True,
# ).add_to(m)



# # markers

# mc = MarkerCluster().add_to(m)


# for id, row in df_cluster.iterrows():
    
#     if row['accident_severity'] == 3:
#         folium.Marker(
#             location = [row['latitude'], row['longitude']],
#             popup = 'Slight severity accident',
#             icon = folium.Icon(color='blue', icon='circle'),
#         ).add_to(mc)

#     elif row['accident_severity'] == 2:
#         folium.Marker(
#             location = [row['latitude'], row['longitude']],
#             popup = 'Serious accident',
#             icon = folium.Icon(color='purple', icon='circle'),
#         ).add_to(mc)
        
#     else:
#         folium.Marker(
#             location = [row['latitude'], row['longitude']],
#             popup = 'Fatal accident',
#             icon = folium.Icon(color='red', icon='remove-sign'),
#         ).add_to(mc)

# m

In [ ]:
# London area heatmap

# map parameters

# m = folium.Map(location=[51.5,-0.1], zoom_start=10, tiles ='cartodbpositron')
# HeatMap(data=df[['latitude', 'longitude']], radius=13).add_to(m)


# # fullscreen feature

# folium.plugins.Fullscreen(
#     position="topright",
#     title="Expand",
#     title_cancel="Exit",
#     force_separate_button=True,
# ).add_to(m)

# m

In [ ]:

# Districts: https://geoportal.statistics.gov.uk/search?q=Local%20Authority%20Districts%20(December%202023)%20Boundaries%20UK%20BFC&sort=Title%7Ctitle%7Casc
# Population estimates: https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/estimatesofthepopulationforenglandandwales

# GEOJSON:
# https://www.data.gov.uk/dataset/6ae5c1ea-b3b6-48f9-adb2-f296e57d3361/local-authority-districts-december-2023-boundaries-uk-bfc

# cp.geojson.data['features'] --> properties: LAD23CD, LAD23NM, FID


In [ ]:
# # England and Wales accident frequency across districts map

# # setting up map properties
# m = folium.Map(location=[54,-1],
#                zoom_start=6,
#                tiles='cartodbpositron',
#                min_zoom=5,
#                min_lat=49.5,
#                max_lat=61,
#                min_lon=-10,
#                max_lon=3,
#                max_bounds=True
#               )

# # fullscreen feature
# folium.plugins.Fullscreen(
#     position="topright",
#     title="Expand",
#     title_cancel="Exit",
#     force_separate_button=True,
# ).add_to(m)


# # retrieve GeoJSON -- online version not stable enough, provided file instead

# # uk_districts = requests.get(
# #     'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/Local_Authority_Districts_December_2023_Boundaries_UK_BFC/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'
# # ).json()


# # creating choropleth map
# cp = folium.Choropleth(
#     geo_data='data/Local_Authority_Districts_December_2023_Boundaries_UK_BFC.geojson', # uk_districts
#     data=population_all_ages,
#     columns=['code','accidents_per_district'],
#     key_on='feature.properties.LAD23CD',
#     bins=13,
#     smooth_factor=2,
#     fill_opacity=1,
#     line_weight=0.3,
#     legend_name='Accident Frequency',
#     highlight=True,
#     nan_fill_color='grey',
#     nan_fill_opacity=0.2,
#     fill_color='Reds' # YlOrRd, BuPu
# ).add_to(m)


# # setting up tooltips

# # index data to LAD23CD codes
# population_all_ages_indexed = population_all_ages.set_index('code')
# population_all_ages_indexed['accidents_per_district'] = population_all_ages_indexed['accidents_per_district'].astype(float)

# # loop through geojson and add accident ratio to properties
# for row in cp.geojson.data['features']:
#     try:
#         row['properties']['accidents_per_district'] = population_all_ages_indexed.loc[row['properties']['LAD23CD'], 'accidents_per_district']
#     except KeyError:
#         row['properties']['accidents_per_district'] = 'No data'

# # customising tooltips
# folium.GeoJsonTooltip(
#     fields=['LAD23NM', 'accidents_per_district'],
#     aliases=['District:', 'Accident frequency:']
# ).add_to(cp.geojson)

# m

### Redundant Geolocator Code below

In [ ]:
# from geopy.geocoders import Nominatim

# df_loc[['latitude']] = df_loc[['latitude']].apply(lambda series: series.apply(lambda value: f'{value:.6f}'))
# df_loc[['longitude']] = df_loc[['longitude']].apply(lambda series: series.apply(lambda value: f'{value:.6f}'))

# converts lat + long in correct format to avoid errors caused by 0 disappearing from the end or scientific format

In [ ]:
#test cell
# df_loc['latitude'][80099]
# previously problematic rows: 2186, 3921, 5625, 5851

In [ ]:
# df_loc['district'] = df_loc['latitude'] + ', ' + df_loc['longitude']
# df_loc['district'][10000]

In [ ]:
# geolocator = Nominatim(user_agent="accident_locator") or GoogleV3(api_key="")
# geolocator = Nominatim(user_agent="accident_locator")

# def find_location(coordinates):
    
#     try:
#         location = geolocator.reverse(coordinates, exactly_one=True)
#         return location.raw['address']['state_district']
    
#     except:
#         return 'Coordinates could not be processed.'

In [ ]:
# %%time

# testing reverse geocoding on part of the dataframe

# x = 0 # row test value
# y = 10 # row test value 2

# df_loc['district'].loc[x:y] = df_loc['district'].loc[x:y].map(find_location)
# df_loc['district'].loc[x:y]


In [ ]:
# df_loc['district'].loc[x:y].value_counts()

In [ ]:
# SEPARATE & INSPECT UNSUCCESSFUL CONVERSION

# df_incorrect_coords = df_loc.copy()
# df_incorrect_coords = df_incorrect_coords[df_incorrect_coords['district'] == 'Coordinates could not be processed.']
# df_incorrect_coords

# checking incorrect coordinates